In [1]:
import json
import os
import pickle
import time
import random
from contextlib import contextmanager
from typing import List, NoReturn, Optional, Tuple, Union

import faiss
import numpy as np
import pandas as pd
from datasets import Dataset, concatenate_datasets, load_from_disk
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm

from retrieval import SparseRetrieval

seed = 2024
random.seed(seed) # python random seed 고정
np.random.seed(seed) # numpy random seed 고정

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 데이터 호출

In [2]:
import pyarrow as pa

file_path_train = '/data/ephemeral/level2-nlp-mrc-nlp-01/data/train_dataset/train/dataset.arrow'

## test1

In [3]:
with open(file_path_train, 'rb') as f:
    reader = pa.ipc.open_stream(f)
    table = reader.read_all()

df_train = table.to_pandas()

In [4]:
df_train.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


In [5]:
import json

file_path_wiki = '/data/ephemeral/level2-nlp-mrc-nlp-01/data/wikipedia_documents.json'

with open(file_path_wiki, 'r') as file:
    data = json.load(file)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
args={'dataset_name':"../data/train_dataset", 
      "model_name_or_path":"bert-base-multilingual-cased",
      "data_path":"../data", "context_path":"wikipedia_documents.json", "use_faiss":False}

In [8]:
from datasets import Dataset, concatenate_datasets, load_from_disk

org_dataset = load_from_disk(args['dataset_name'])
full_ds = concatenate_datasets(
    [
        org_dataset["train"].flatten_indices(),
        org_dataset["validation"].flatten_indices(),
    ]
)  # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*" * 40, "query dataset", "*" * 40)
print(full_ds)

**************************************** query dataset ****************************************
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 4192
})


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(args['model_name_or_path'], use_fast=False,)

tokenize_fn=tokenizer.tokenize
data_path=args['data_path']
context_path=args['context_path']


NameError: name 'args' is not defined

In [36]:
with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)

contexts = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)
print(f"Lengths of unique contexts : {len(contexts)}")
ids = list(range(len(contexts)))

tfidfv = TfidfVectorizer(
    tokenizer=tokenize_fn, ngram_range=(1, 2), max_features=50000,
)

Lengths of unique contexts : 56737


In [37]:
# Pickle을 저장합니다.
pickle_name = f"sparse_embedding.bin"
tfidfv_name = f"tfidv.bin"
emd_path = os.path.join(data_path, pickle_name)
tfidfv_path = os.path.join(data_path, tfidfv_name)

if os.path.isfile(emd_path) and os.path.isfile(tfidfv_path):
    with open(emd_path, "rb") as file:
        p_embedding = pickle.load(file)
    with open(tfidfv_path, "rb") as file:
        tfidfv = pickle.load(file)
    print("Embedding pickle load.")
else:
    print("Build passage embedding")
    p_embedding = tfidfv.fit_transform(contexts)
    print(p_embedding.shape)
    with open(emd_path, "wb") as file:
        pickle.dump(p_embedding, file)
    with open(tfidfv_path, "wb") as file:
        pickle.dump(tfidfv, file)
    print("Embedding pickle saved.")

Embedding pickle load.


In [48]:
def get_relevant_score(
    queries: List, k: Optional[int] = 1
) -> Tuple[List, List]:
    query_vec = tfidfv.transform(queries)
    assert (
        np.sum(query_vec) != 0
    ), "오류가 발생했습니다. 이 오류는 보통 query에 vectorizer의 vocab에 없는 단어만 존재하는 경우 발생합니다."

    result = query_vec * p_embedding.T
    if not isinstance(result, np.ndarray):
        result = result.toarray()
    doc_scores = []
    doc_indices = []
    for i in range(result.shape[0]):
        sorted_result = np.argsort(result[i, :])[::-1]
        doc_scores.append(result[i, :][sorted_result].tolist()[:k])
        doc_indices.append(sorted_result.tolist()[:k])
    return doc_scores, doc_indices

In [49]:
doc_scores, doc_indices=get_relevant_score(full_ds['question'])

AttributeError: 'BertTokenizer' object has no attribute 'split_special_tokens'

# re

In [3]:
from transformers import AutoTokenizer

args={'dataset_name':"../data/train_dataset", 
      "model_name_or_path":"domyoung/squad-test",
      "data_path":"../data", "context_path":"wikipedia_documents.json", "use_faiss":False}

tokenizer = AutoTokenizer.from_pretrained(args['model_name_or_path'], use_fast=False,)

tokenize_fn=tokenizer.tokenize
data_path=args['data_path']
context_path=args['context_path']


In [4]:
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f"[{name}] done in {time.time() - t0:.3f} s")

In [5]:
# Test sparse
org_dataset = load_from_disk(args['dataset_name'])
full_ds = concatenate_datasets(
    [
        org_dataset["train"].flatten_indices(),
        org_dataset["validation"].flatten_indices(),
    ]
)  # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*" * 40, "query dataset", "*" * 40)
print(full_ds)

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(args['model_name_or_path'], use_fast=False,)

retriever = SparseRetrieval(
    tokenize_fn=tokenizer.tokenize,
    data_path=args['data_path'],
    context_path=args['context_path'],
)


**************************************** query dataset ****************************************
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 4192
})
Lengths of unique contexts : 56737


In [6]:
retriever.get_sparse_embedding()

Embedding pickle load.


In [7]:
doc_scores, doc_indices = retriever.get_relevant_doc_bulk(full_ds["question"])
doc_scores, doc_indices=[x[0] for x in doc_scores],[x[0] for x in doc_indices]

In [8]:
len(full_ds["question"])

4192

In [9]:
len(doc_scores)

4192

In [26]:
import json

with open("../data/wikipedia_documents.json") as f:
    wiki_json = json.load(f)

In [27]:
with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)

contexts = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)  # set 은 매번 순서가 바뀌므로

In [28]:
with timer("bulk query by exhaustive search"):
    df = retriever.retrieve(full_ds)
    df["correct"] = df["original_context"] == df["context"]
    print(
        "correct retrieval result by exhaustive search",
        df["correct"].sum() / len(df),
    )
df['scores']=doc_scores


[query exhaustive search] done in 34.676 s


Sparse retrieval: 100%|██████████| 4192/4192 [00:00<00:00, 8961.23it/s]

correct retrieval result by exhaustive search 0.2366412213740458
[bulk query by exhaustive search] done in 35.157 s


In [40]:
pd.options.display.max_rows = 100

- q_value 넣었을 때 p_value return 해야 함
- 임베딩 하기 전에 /n, 특수기호, 외국어 등 제거하는 작업
- 

In [67]:
df_test=df[df['correct']==False].sort_values("scores",ascending=False)
print(df_test.iloc[3,3])
t1=df_test.iloc[3,3].replace('\\n','')
print(t1)
t1[df_test.iloc[3,4]['answer_start'][0]]

;두명의 연인(二人の恋人)\n:마코토와 세카이가 애정을 나누기 위해 마코토의 집으로 왔는데, 그곳에는 이미 코토노하가 먼저 와있는 상태. 마코토의 방에서 몸을 겹치던 둘앞에 갑자기 나타난 코토노하, 그녀는 세카이가 아랑곳 않고 태연히 마코토와 몸을 겹치고 비상식적 상황에 경악하면서도 세카이도 지지않고 마코토와 몸을 겹친다. 다음날 점심, 오전 교실에서 코토노하와 함께 셋이서 하자는 말에 반발하면서도 찾아간 옥상. 이미 코토노하와 마코토가 먼저 몸을 겹치고 있었고 세카이가 오자 코토노하는 마코토에게 누구 먼저 넣고 싶은지를 묻는다. 세카이를 선택하는 마코토. 마코토와 몸을 겹치던 세카이는 코토노하와 자신을 공평히 사랑해 준다면 이 비상식적 상황을 받아들이기로 하고, 옆에서 둘을 보며 스스로를 위로하던 코토노하와 진한 키스를 나눈다. 시간이 흘러 크리스마스 이브. 길을 걷던 세카이와 코토노하는 마코토에게 줄 크리스마스 선물 이야기를 즐겁게 나누고, 둘이 마코토에게 선사할 선물이 같음을 알게 되면서 마코토가 보일 반응을 기대하게 된다. 순간 하늘에선 눈이 내리기 시작하는데... PS2판의 경우엔 화면상으로 보이는 애정 행위 수준이 키스나 꽉 안는정도까지만 표현되었다. 다만, 크리스마스를 앞두고 나누는 대화 중 크리스마스 선물의 경우 PC판과 동일\n;안녕, 세카이(さよなら世界)\n:세츠나와 마코토는 세카이의 파리행을 단념 시키고자 결국 마코토의 방에서 관계를 가지는 연기를 한다. 세츠나는 이 과정을 핸드폰을 통해 세카이에게 상황 전달하게 되나 참지 못한 세카이는 전화를 끊어버리게 되고 연기는 실제로 끝까지 진행되게 된다. 그 후 세카이는 결국 어머니와 파리에 가게 되고 마코토와 세츠나, 나나미들과의 사이는 자연히 소원해진다. 이후 여자들과 적당히 노는 생활을 반복해가는 마코토. PS2판의 경우엔 관계를 가지는 연기의 수준이 키스 정도까지로 수정되었다.
;두명의 연인(二人の恋人):마코토와 세카이가 애정을 나누기 위해 마코토의 집으로 왔는데, 그곳에는 이미 코토노하가 

'스'

In [68]:
len('鮮')

1

In [59]:
temp[df_test.iloc[5,4]['answer_start'][0]]

'러'

In [41]:
df[df['correct']==False].sort_values("scores",ascending=False).iloc[:100]

,question,id,context,original_context,answers,correct,scores
1086,티타노케라톱스가 펜타케라톱스보다 더 유사한 종은?,mrc-0-000362,티타노케라톱스는 원래 펜타케라톱스로 분류되었으나 롱리치가 그 특징들이 펜타케라톱스와...,"펜타케라톱스(Pentaceratops, 다섯 개의 뿔이 달린 얼굴)는 백악기 후기 ...","{'answer_start': [408], 'text': ['트리케라톱스']}",False,0.836181
735,세르비아 몬테네그로의 전 국호는?,mrc-0-001921,"신유고슬라비아는 2003년 2월 4일, 국호를 '세르비아 몬테네그로 국가연합'으로 ...","신유고슬라비아는 2003년 2월 4일, 국호를 '세르비아 몬테네그로 국가연합'으로 ...","{'answer_start': [0], 'text': ['신유고슬라비아']}",False,0.729887
1132,글리세르알데하이드 3-인산이 생성되는 반응은?,mrc-1-000892,"먼저, 글리세르알데하이드 3-인산 탈수소효소의 활성 부위에서 시스테인 잔기는 글리세...","삼탄당 인산 이성질화효소의 반응 메커니즘은 ""엔올""의 중간생성물 형성을 포함한다. ...","{'answer_start': [492], 'text': ['해당과정']}",False,0.675025
369,세카이와 코토노하가 마코토에게 줄 선물은 언제를 위한 것인가?,mrc-0-003610,;선혈의 결말(鮮血の結末)\n:아침 전차와 학교에서 마코토에게로의 접근이 나나미와 ...,;두명의 연인(二人の恋人)\n:마코토와 세카이가 애정을 나누기 위해 마코토의 집으로...,"{'answer_start': [456], 'text': ['크리스마스']}",False,0.647834
2112,갈릴레오 갈릴레이가 쓴 책 중 출판으로 인해 감옥에 갇혔던 저서의 제목은?,mrc-0-000856,"갈릴레오 갈릴레이란 이름은 갈릴레오 보냐우티(Galileo Bonaiuti, 137...","갈릴레오 갈릴레이(Galileo Galilei, 1564년 2월 15일 ~ 1642...","{'answer_start': [478], 'text': ['〈천문학 대화〉']}",False,0.637152
1496,비트겐슈타인이 불만을 가진 사람은?,mrc-0-004496,비트겐슈타인은 1911년 8월부터 버트런드 러셀의 강의실에 출석하였고 얼마 지나지 ...,비트겐슈타인은 1911년 8월부터 버트런드 러셀의 강의실에 출석하였고 얼마 지나지 ...,"{'answer_start': [347], 'text': ['러셀']}",False,0.628227
2143,비트겐슈타인이 오지에서 독거하며 써낸 작품은?,mrc-0-004617,비트겐슈타인은 1911년 8월부터 버트런드 러셀의 강의실에 출석하였고 얼마 지나지 ...,비트겐슈타인은 1911년 8월부터 버트런드 러셀의 강의실에 출석하였고 얼마 지나지 ...,"{'answer_start': [1302], 'text': ['《논리 철학 논고》']}",False,0.616701
1024,크렘린 궁 석벽의 현재 용도는?,mrc-0-001197,소비에트 정부는 정부 기관들을 1918년 3월 12일에 페트로그라드(상트페테르부르크...,소비에트 정부는 정부 기관들을 1918년 3월 12일에 페트로그라드(상트페테르부르크...,"{'answer_start': [266], 'text': ['혁명영웅들의 묘지']}",False,0.616073
33,비트겐슈타인은 누구와 친구가 되었는가?,mrc-0-004050,제1차 세계대전이 일어난 1914년에 비트겐슈타인은 1년째 은둔 생활하고 있었다. ...,몽크의 전기에 따르면 비트겐슈타인은 전쟁이 시작되었는데도 철학 강의나 하고 있는 스...,"{'answer_start': [606], 'text': ['케이스 커크']}",False,0.599398
1362,볼셰비키와 멘셰비키가 갈라선 해는?,mrc-0-004119,러시아 혁명을 전후하여 활동한 우크라이나의 아나키스트들은 혁명 초기에는 볼셰비키에 ...,제정러시아의 중앙아시아에 위치한 비슈케크에서 루마니아 출신 농민의 아들로 태어났다....,"{'answer_start': [90], 'text': ['1903년']}",False,0.593714


In [32]:
df[df['correct']==True].sort_values("scores",ascending=True)

,question,id,context,original_context,answers,correct,scores
2896,불타는 수도원에서 형제들이 찾아냈지만 그 이후 볼 수 없었던 것은?,mrc-0-003902,이 수도원은 삼위일체에게 봉헌되었고 1265년에 내부 장식을 완성하였다. 1263년...,이 수도원은 삼위일체에게 봉헌되었고 1265년에 내부 장식을 완성하였다. 1263년...,"{'answer_start': [349], 'text': ['성물']}",True,0.115759
1115,방실이가 미군 부대에서 처음으로 활동하기 시작한 해는?,mrc-0-004314,방실이는 1963년 10월 29일에 인천광역시 강화에서 태어났다. 강화여자고등학교를...,방실이는 1963년 10월 29일에 인천광역시 강화에서 태어났다. 강화여자고등학교를...,"{'answer_start': [54], 'text': ['1982년']}",True,0.119769
4086,돈대에서 무너져내린 포좌부분이 있는 위치는?,mrc-1-000658,"건평 돈대는 朝鮮 肅宗 5년 강화유수 尹以濟가 축조한 것으로, 굴암돈대 북쪽 1,4...","건평 돈대는 朝鮮 肅宗 5년 강화유수 尹以濟가 축조한 것으로, 굴암돈대 북쪽 1,4...","{'answer_start': [143], 'text': ['서남쪽']}",True,0.120150
1049,인정식이 입지를 넓히는데 걸림돌이 된 그의 사상은?,mrc-1-001703,1938년 4월 공화계 야학 사건의 주모자로 검거되었다. 공화계 야학은 인정식의 고...,1938년 4월 공화계 야학 사건의 주모자로 검거되었다. 공화계 야학은 인정식의 고...,"{'answer_start': [393], 'text': ['사회주의']}",True,0.122970
3450,"학교 지휘부가 ""혼란을 피하여 침착하게 한강을 넘어라""라고 지시를 내린 이유는?",mrc-0-003333,"1950년 6월 25일 새벽 4시경, 조선민주주의 인민공화국의 대대적인 남침으로 인...","1950년 6월 25일 새벽 4시경, 조선민주주의 인민공화국의 대대적인 남침으로 인...","{'answer_start': [875], 'text': ['생도들의 전멸']}",True,0.129390
...,...,...,...,...,...,...,...
2660,웅게른슈테른베르크가 포로로 붙잡혔던 유격대의 리더는 누구였나요?,mrc-0-001855,볼셰비키들은 10월 혁명 직후부터 몽골 침투를 시작했다. 1921년이 되면 러시아 ...,볼셰비키들은 10월 혁명 직후부터 몽골 침투를 시작했다. 1921년이 되면 러시아 ...,"{'answer_start': [1784], 'text': ['표트르 슈체틴킨']}",True,0.635495
946,시에라 네바다 산맥에서 할 수 있는 스포츠는?,mrc-0-003861,"스페인은 피레네 산맥, 시스테마 센트럴 산맥, 세스테마 이베리코 산맥, 시에라 네바...","스페인은 피레네 산맥, 시스테마 센트럴 산맥, 세스테마 이베리코 산맥, 시에라 네바...","{'answer_start': [309], 'text': ['스키']}",True,0.641959
3721,방어력이 다른 타입보다 좋은 포켓몬은?,mrc-0-001794,"《포켓몬스터 금·은》은 포켓몬의 포획과 대결, 진화와 같은 기본적인 구조를 《포켓몬...","《포켓몬스터 금·은》은 포켓몬의 포획과 대결, 진화와 같은 기본적인 구조를 《포켓몬...","{'answer_start': [780], 'text': ['강철 타입 포켓몬']}",True,0.656969
2901,어셈블리의 구성 요소 중 어셈블리에 대한 고유값을 가진 것은?,mrc-0-002168,닷넷 어셈블리\n\n중간 언어인 MSIL 코드는 닷넷 어셈블리에 저장된다. 닷넷 어...,닷넷 어셈블리\n\n중간 언어인 MSIL 코드는 닷넷 어셈블리에 저장된다. 닷넷 어...,"{'answer_start': [260], 'text': ['공개 키 암호 방식(p...",True,0.662235
